In [1]:
import pandas as pd
import urllib.request
import json
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable
import time
import numpy as np  
import folium


In [2]:
path = r"D:\OneDrive\energia_lab\data\\"

In [3]:
url = 'https://dadosabertos.aneel.gov.br/dataset/f9336ee6-7562-4c92-880a-d7217df6cc94/resource/28686698-544b-48e2-af58-ad4f7616246c/download/projetos-eficiencia-energetica-empresa.csv'
df = pd.read_csv(url, encoding='ISO-8859-1', sep=';', on_bad_lines='skip', header=0)

In [4]:
df['DatGeracaoConjuntoDados'] = pd.to_datetime(df['DatGeracaoConjuntoDados'])
df['DatInicioProjeto'] = pd.to_datetime(df['DatInicioProjeto'])
df['DatConclusaoProjeto'] = pd.to_datetime(df['DatConclusaoProjeto'])

In [5]:
df['VlrRetiradaDemandaPontaTotal'] = df['VlrRetiradaDemandaPontaTotal'].str.replace(',', '.').astype(float)
df['VlrRcbGlobal'] = df['VlrRcbGlobal'].str.replace(',', '.').astype(float)
df['VlrEnergiaEconomizadaTotal'] = df['VlrEnergiaEconomizadaTotal'].str.replace(',', '.').astype(float)
df['VlrCustoTotal'] = df['VlrCustoTotal'].str.replace(',', '.').astype(float)

In [6]:
df = df[df['VlrCustoTotal'] != 0]

In [7]:
# Define a small epsilon value to prevent division by zero
epsilon = 0.01

# Recalculate the score with epsilon added to the denominator to avoid division by zero
df['Score'] = (df['VlrEnergiaEconomizadaTotal'] + df['VlrRetiradaDemandaPontaTotal']) / (df['VlrCustoTotal'] + epsilon)

In [8]:
# Drop rows where 'Score' is null
df = df.dropna(subset=['Score'])

In [9]:
# Sort df by 'Score' in descending order
df = df.sort_values('Score', ascending=False)

In [10]:
# Calculate the 50th percentile of 'Score'
score_95th_percentile = df['Score'].quantile(0.95)

# Filter df to only include rows where 'Score' is above the 50th percentile
df_above_95 = df[df['Score'] > score_95th_percentile]

In [11]:
df = df_above_95

In [12]:
# Rescale 'Score' to range from 1 to 10
df['Score'] = 1 + (df['Score'] - df['Score'].min()) * (10 - 1) / (df['Score'].max() - df['Score'].min())

In [13]:
df['NomAgente'] = df['NomAgente'].str.upper()

In [14]:
# Instanciando o geocodificador com um timeout maior
geolocator = Nominatim(user_agent="unique_user_agent", timeout=10)

# Função para obter endereço e coordenadas com retry
def get_location(name, cache):
    retries = 3
    for _ in range(retries):
        if name in cache:
            return cache[name]
        try:
            location = geolocator.geocode(name)
            if location:
                result = (location.address, location.latitude, location.longitude)
            else:
                result = (None, None, None)
            cache[name] = result
            return result
        except (GeocoderTimedOut, GeocoderUnavailable) as e:
            print(f"Retry for {name} due to error: {str(e)}")
            time.sleep(5)  # Espera antes de tentar novamente
    return None, None, None

# Função para processar os nomes únicos e preencher as informações de localização
def process_names(names, delay=2):
    location_cache = {}
    results = {'address': [], 'latitude': [], 'longitude': []}
    for name in names:
        address, lat, lon = get_location(name, location_cache)
        results['address'].append(address)
        results['latitude'].append(lat)
        results['longitude'].append(lon)
        time.sleep(delay)  # Delay para respeitar os limites da API
    return results, location_cache

# Extrair nomes únicos de agentes
nomes_unicos = df['NomAgente'].unique()

# Processar os nomes para obter as informações de localização
location_results, cache = process_names(nomes_unicos)

# Adicionando os resultados ao DataFrame
df['Address'] = df['NomAgente'].map({name: addr for name, (addr, _, _) in cache.items()})
df['Latitude'] = df['NomAgente'].map({name: lat for name, (_, lat, _) in cache.items()})
df['Longitude'] = df['NomAgente'].map({name: lon for name, (_, _, lon) in cache.items()})

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 192 entries, 1219 to 2041
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   DatGeracaoConjuntoDados       192 non-null    datetime64[ns]
 1   NomAgente                     192 non-null    object        
 2   IdeEmpresaProponenteProjeto   192 non-null    int64         
 3   DscCodProjeto                 192 non-null    object        
 4   DscTituloProjeto              192 non-null    object        
 5   DscTipologia                  192 non-null    object        
 6   VlrCustoTotal                 192 non-null    float64       
 7   VlrRcbGlobal                  113 non-null    float64       
 8   VlrEnergiaEconomizadaTotal    192 non-null    float64       
 9   VlrRetiradaDemandaPontaTotal  192 non-null    float64       
 10  DscObjetivo                   28 non-null     object        
 11  DscJustificativa              103

In [16]:
# Set pandas display options
pd.set_option('display.max_rows', None)

# Display value counts for 'NomAgente'
print(df['NomAgente'].value_counts(dropna=False))

NomAgente
CEMIG DISTRIBUIÇÃO S.A                                             18
LIGHT SERVIÇOS DE ELETRICIDADE S A                                 14
COPEL DISTRIBUICAO S.A.                                            11
ENERGISA MATO GROSSO DO SUL - DISTRIBUIDORA DE ENERGIA S.A.        11
COMPANHIA PAULISTA DE FORÇA E LUZ                                   9
COMPANHIA ESTADUAL DE DISTRIBUIÇÃO DE ENERGIA ELÉTRICA - CEEE-D     9
ESPÍRITO SANTO DISTRIBUIÇÃO DE ENERGIA S/A.                         8
EQUATORIAL MARANHAO DISTRIBUIDORA DE ENERGIA S.A.                   8
ELEKTRO REDES S.A.                                                  7
ENERGISA MATO GROSSO - DISTRIBUIDORA DE ENERGIA S.A.                7
AMPLA ENERGIA E SERVIÇOS S.A.                                       6
RGE SUL DISTRIBUIDORA DE ENERGIA S.A.                               6
COMPANHIA DE ELETRICIDADE DO ESTADO DA BAHIA                        5
ENERGISA SERGIPE - DISTRIBUIDORA DE ENERGIA S.A                     5
CEB DISTRI

In [17]:
df['Address'].value_counts(dropna=False)

Address
None                                                                                                                                                                                                                                                                                       140
Copel Distribuição S.A., Rua Vicente Machado, Guarapuava, Região Geográfica Imediata de Guarapuava, Região Geográfica Intermediária de Guarapuava, Paraná, Região Sul, 85010-050, Brasil                                                                                                    12
Companhia Paulista de Força e Luz, Avenida Sete de Setembro, Vila Pavani, Nova Granada, Região Imediata de São José do Rio Preto, Região Metropolitana de São José do Rio Preto, Região Geográfica Intermediária de São José do Rio Preto, São Paulo, Região Sudeste, 15440-000, Brasil      9
EDP - Espírito Santo Distribuição de Energia S. A., 14, Rua Sergipe, Centro, Mucurici, Região Geográfica Imediata de São Mateus, Re

In [18]:
# Count null values in 'Address', 'Latitude', and 'Longitude'
address_null_count = df['Address'].isnull().sum()
latitude_null_count = df['Latitude'].isnull().sum()
longitude_null_count = df['Longitude'].isnull().sum()

print(f"Null count in 'Address': {address_null_count}")
print(f"Null count in 'Latitude': {latitude_null_count}")
print(f"Null count in 'Longitude': {longitude_null_count}")

Null count in 'Address': 140
Null count in 'Latitude': 140
Null count in 'Longitude': 140


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 192 entries, 1219 to 2041
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   DatGeracaoConjuntoDados       192 non-null    datetime64[ns]
 1   NomAgente                     192 non-null    object        
 2   IdeEmpresaProponenteProjeto   192 non-null    int64         
 3   DscCodProjeto                 192 non-null    object        
 4   DscTituloProjeto              192 non-null    object        
 5   DscTipologia                  192 non-null    object        
 6   VlrCustoTotal                 192 non-null    float64       
 7   VlrRcbGlobal                  113 non-null    float64       
 8   VlrEnergiaEconomizadaTotal    192 non-null    float64       
 9   VlrRetiradaDemandaPontaTotal  192 non-null    float64       
 10  DscObjetivo                   28 non-null     object        
 11  DscJustificativa              103

In [20]:
df['DscTipologia'].value_counts(dropna=False)

DscTipologia
Industrial                     40
Poder Público                  34
Pelo Lado da Oferta/Perdas     29
Residencial                    19
Comércio e Serviços            16
Educacional                    15
Iluminação Pública             15
Baixa Renda                    13
Serviços Públicos               7
Diagnóstico Energético          2
Rural                           1
Gestão Energética Municipal     1
Name: count, dtype: int64

In [21]:
df_null_address = df[df['Address'].isnull()]

In [22]:
df_null_address.info()

<class 'pandas.core.frame.DataFrame'>
Index: 140 entries, 2069 to 2041
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   DatGeracaoConjuntoDados       140 non-null    datetime64[ns]
 1   NomAgente                     140 non-null    object        
 2   IdeEmpresaProponenteProjeto   140 non-null    int64         
 3   DscCodProjeto                 140 non-null    object        
 4   DscTituloProjeto              140 non-null    object        
 5   DscTipologia                  140 non-null    object        
 6   VlrCustoTotal                 140 non-null    float64       
 7   VlrRcbGlobal                  84 non-null     float64       
 8   VlrEnergiaEconomizadaTotal    140 non-null    float64       
 9   VlrRetiradaDemandaPontaTotal  140 non-null    float64       
 10  DscObjetivo                   24 non-null     object        
 11  DscJustificativa              75 

In [23]:
df_null_address['NomAgente'].value_counts(dropna=False)

NomAgente
CEMIG DISTRIBUIÇÃO S.A                                             18
LIGHT SERVIÇOS DE ELETRICIDADE S A                                 14
ENERGISA MATO GROSSO DO SUL - DISTRIBUIDORA DE ENERGIA S.A.        11
COMPANHIA ESTADUAL DE DISTRIBUIÇÃO DE ENERGIA ELÉTRICA - CEEE-D     9
EQUATORIAL MARANHAO DISTRIBUIDORA DE ENERGIA S.A.                   8
ENERGISA MATO GROSSO - DISTRIBUIDORA DE ENERGIA S.A.                7
RGE SUL DISTRIBUIDORA DE ENERGIA S.A.                               6
AMPLA ENERGIA E SERVIÇOS S.A.                                       6
ENERGISA SERGIPE - DISTRIBUIDORA DE ENERGIA S.A                     5
CEB DISTRIBUIÇÃO S.A.                                               5
COMPANHIA ENERGÉTICA DO RIO GRANDE DO NORTE COSERN                  4
ENERGISA MATO GROSSO DISTRIBUIDORA DE ENERGIA S.A.                  4
CENTRAIS ELÉTRICAS DO PARÁ S.A. - CELPA                             4
RIO GRANDE ENERGIA SA                                               4
EQUATORIAL

In [24]:
import pandas as pd

# Lista de estados e capitais brasileiras
estados_capitais = [
    ("ACRE", "RIO BRANCO"),
    ("ALAGOAS", "MACEIÓ"),
    ("AMAPÁ", "MACAPÁ"),
    ("AMAZONAS", "MANAUS"),
    ("BAHIA", "SALVADOR"),
    ("CEARÁ", "FORTALEZA"),
    ("DISTRITO FEDERAL", "BRASÍLIA"),
    ("ESPÍRITO SANTO", "VITÓRIA"),
    ("GOIÁS", "GOIÂNIA"),
    ("MARANHÃO", "SÃO LUÍS"),
    ("MATO GROSSO", "CUIABÁ"),
    ("MATO GROSSO DO SUL", "CAMPO GRANDE"),
    ("MINAS GERAIS", "BELO HORIZONTE"),
    ("PARÁ", "BELÉM"),
    ("PARAÍBA", "JOÃO PESSOA"),
    ("PARANÁ", "CURITIBA"),
    ("PERNAMBUCO", "RECIFE"),
    ("PIAUÍ", "TERESINA"),
    ("RIO DE JANEIRO", "RIO DE JANEIRO"),
    ("RIO GRANDE DO NORTE", "NATAL"),
    ("RIO GRANDE DO SUL", "PORTO ALEGRE"),
    ("RONDÔNIA", "PORTO VELHO"),
    ("RORAIMA", "BOA VISTA"),
    ("SANTA CATARINA", "FLORIANÓPOLIS"),
    ("SÃO PAULO", "SÃO PAULO"),
    ("SERGIPE", "ARACAJU"),
    ("TOCANTINS", "PALMAS")
]

# Criando o DataFrame simulado com os nomes das empresas
data = {
    "NomAgente": [
        "CEMIG DISTRIBUIÇÃO S.A.",
        "ENERGISA SUL-SUDESTE DISTRIBUIDORA DE ENERGIA S.A.",
        "COMPANHIA PIRATININGA DE FORÇA E LUZ",
        "ENERGISA MATO GROSSO DISTRIBUIDORA DE ENERGIA S A",
        "RGE SUL DISTRIBUIDORA DE ENERGIA S.A.",
        "CEB DISTRIBUIÇÃO SA",
        "EDP ESPIRITO SANTO DISTRIBUIDORA DE ENERGIA S/A",
        "CENTRAIS ELÉTRICAS DE RONDÔNIA S.A   CERON",
        "COMPANHIA PAULISTA DE FORA E LUZ",
        "ENERGISA MATO GROSSO DO SUL - DISTRIBUIDORA DE ENERGIA S.A.",
        "CEB DISTRIBUIÇÃO S/A",
        "ENERGISA SERGIPE DISTRIBUIDORA DE ENERGIA S.A",
        "NEOENERGIA DISTRIBUIÇÃO BRASÍLIA S.A",
        "ENERGISA PARABA DISTRIBUIDORA DE ENERGIA S.A",
        "COMPANHIA ENERGETICA DE ALAGOAS",
        "COMPANHIA ENERGÉTICA DE ALAGOAS",
        "ENERGISA ACRE DISTRIBUIDORA DE ENERGIA S.A."
    ]
}

df1 = pd.DataFrame(data)

# Função para encontrar estado e capital baseado no nome da empresa
def find_state_and_capital(nom_agente):
    for estado, capital in estados_capitais:
        if estado in nom_agente.upper():
            return (estado, capital)
        elif capital in nom_agente.upper():
            return (estado, capital)
    return ("", "")

# Aplicar a função em cada linha do DataFrame
df1["Estado"], df1["Capital"] = zip(*df1["NomAgente"].apply(find_state_and_capital))

df1


,NomAgente,Estado,Capital
0,CEMIG DISTRIBUIÇÃO S.A.,,
1,ENERGISA SUL-SUDESTE DISTRIBUIDORA DE ENERGIA ...,,
2,COMPANHIA PIRATININGA DE FORÇA E LUZ,,
3,ENERGISA MATO GROSSO DISTRIBUIDORA DE ENERGIA S A,MATO GROSSO,CUIABÁ
4,RGE SUL DISTRIBUIDORA DE ENERGIA S.A.,,
5,CEB DISTRIBUIÇÃO SA,,
6,EDP ESPIRITO SANTO DISTRIBUIDORA DE ENERGIA S/A,,
7,CENTRAIS ELÉTRICAS DE RONDÔNIA S.A CERON,RONDÔNIA,PORTO VELHO
8,COMPANHIA PAULISTA DE FORA E LUZ,,
9,ENERGISA MATO GROSSO DO SUL - DISTRIBUIDORA DE...,MATO GROSSO,CUIABÁ


In [25]:
df1['Estado'].value_counts(dropna=False)

Estado
                    9
MATO GROSSO         2
ALAGOAS             2
RONDÔNIA            1
SERGIPE             1
DISTRITO FEDERAL    1
ACRE                1
Name: count, dtype: int64

In [26]:
df1

,NomAgente,Estado,Capital
0,CEMIG DISTRIBUIÇÃO S.A.,,
1,ENERGISA SUL-SUDESTE DISTRIBUIDORA DE ENERGIA ...,,
2,COMPANHIA PIRATININGA DE FORÇA E LUZ,,
3,ENERGISA MATO GROSSO DISTRIBUIDORA DE ENERGIA S A,MATO GROSSO,CUIABÁ
4,RGE SUL DISTRIBUIDORA DE ENERGIA S.A.,,
5,CEB DISTRIBUIÇÃO SA,,
6,EDP ESPIRITO SANTO DISTRIBUIDORA DE ENERGIA S/A,,
7,CENTRAIS ELÉTRICAS DE RONDÔNIA S.A CERON,RONDÔNIA,PORTO VELHO
8,COMPANHIA PAULISTA DE FORA E LUZ,,
9,ENERGISA MATO GROSSO DO SUL - DISTRIBUIDORA DE...,MATO GROSSO,CUIABÁ


In [27]:
# Atualizando o DataFrame com as novas informações fornecidas
updates = {
    "CEMIG DISTRIBUIÇÃO S.A.": ("MINAS GERAIS", "BELO HORIZONTE"),
    "COMPANHIA PIRATININGA DE FORÇA E LUZ": ("SÃO PAULO", "SANTOS"),
    "ENERGISA SUL-SUDESTE DISTRIBUIDORA DE ENERGIA S.A.": ("SÃO PAULO", "PRESIDENTE PRUDENTE"),
    "RGE SUL DISTRIBUIDORA DE ENERGIA S.A.": ("RIO GRANDE DO SUL", "CAXIAS DO SUL"),
    "CEB DISTRIBUIÇÃO SA": ("DISTRITO FEDERAL", "BRASÍLIA"),
    "CEB DISTRIBUIÇÃO S/A": ("DISTRITO FEDERAL", "BRASÍLIA"),
    "EDP ESPIRITO SANTO DISTRIBUIDORA DE ENERGIA S/A": ("ESPÍRITO SANTO", "VITÓRIA"),
    "COMPANHIA PAULISTA DE FORA E LUZ": ("SÃO PAULO", "SOROCABA"),
    "ENERGISA PARABA DISTRIBUIDORA DE ENERGIA S.A": ("PARAÍBA", "JOÃO PESSOA")
}

# Aplicar as atualizações no DataFrame
for agent, (estado, capital) in updates.items():
    df1.loc[df1['NomAgente'] == agent, ['Estado', 'Capital']] = estado, capital



df1


,NomAgente,Estado,Capital
0,CEMIG DISTRIBUIÇÃO S.A.,MINAS GERAIS,BELO HORIZONTE
1,ENERGISA SUL-SUDESTE DISTRIBUIDORA DE ENERGIA ...,SÃO PAULO,PRESIDENTE PRUDENTE
2,COMPANHIA PIRATININGA DE FORÇA E LUZ,SÃO PAULO,SANTOS
3,ENERGISA MATO GROSSO DISTRIBUIDORA DE ENERGIA S A,MATO GROSSO,CUIABÁ
4,RGE SUL DISTRIBUIDORA DE ENERGIA S.A.,RIO GRANDE DO SUL,CAXIAS DO SUL
5,CEB DISTRIBUIÇÃO SA,DISTRITO FEDERAL,BRASÍLIA
6,EDP ESPIRITO SANTO DISTRIBUIDORA DE ENERGIA S/A,ESPÍRITO SANTO,VITÓRIA
7,CENTRAIS ELÉTRICAS DE RONDÔNIA S.A CERON,RONDÔNIA,PORTO VELHO
8,COMPANHIA PAULISTA DE FORA E LUZ,SÃO PAULO,SOROCABA
9,ENERGISA MATO GROSSO DO SUL - DISTRIBUIDORA DE...,MATO GROSSO,CUIABÁ


In [28]:
# Merge df_null_address with df1 on 'NomAgente'
df_null_address = pd.merge(df_null_address, df1[['NomAgente', 'Estado', 'Capital']], on='NomAgente', how='left')

In [29]:
df_null_address.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 21 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   DatGeracaoConjuntoDados       140 non-null    datetime64[ns]
 1   NomAgente                     140 non-null    object        
 2   IdeEmpresaProponenteProjeto   140 non-null    int64         
 3   DscCodProjeto                 140 non-null    object        
 4   DscTituloProjeto              140 non-null    object        
 5   DscTipologia                  140 non-null    object        
 6   VlrCustoTotal                 140 non-null    float64       
 7   VlrRcbGlobal                  84 non-null     float64       
 8   VlrEnergiaEconomizadaTotal    140 non-null    float64       
 9   VlrRetiradaDemandaPontaTotal  140 non-null    float64       
 10  DscObjetivo                   24 non-null     object        
 11  DscJustificativa              75

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 192 entries, 1219 to 2041
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   DatGeracaoConjuntoDados       192 non-null    datetime64[ns]
 1   NomAgente                     192 non-null    object        
 2   IdeEmpresaProponenteProjeto   192 non-null    int64         
 3   DscCodProjeto                 192 non-null    object        
 4   DscTituloProjeto              192 non-null    object        
 5   DscTipologia                  192 non-null    object        
 6   VlrCustoTotal                 192 non-null    float64       
 7   VlrRcbGlobal                  113 non-null    float64       
 8   VlrEnergiaEconomizadaTotal    192 non-null    float64       
 9   VlrRetiradaDemandaPontaTotal  192 non-null    float64       
 10  DscObjetivo                   28 non-null     object        
 11  DscJustificativa              103

In [31]:
# Drop rows with NaN values in 'Latitude' or 'Longitude'
df = df.dropna(subset=['Latitude', 'Longitude'])

# Initialize the geolocator
geolocator = Nominatim(user_agent="geoapiExercises")

def get_location_by_coordinates(lat, lon):
    location = geolocator.reverse([lat, lon], exactly_one=True)
    address = location.raw['address']
    state = address.get('state', '')
    city = address.get('city', '')
    return pd.Series([state, city])

# Apply the function to the DataFrame
df[['Estado', 'Capital']] = df.apply(lambda row: get_location_by_coordinates(row['Latitude'], row['Longitude']), axis=1)

C:\Users\surze\AppData\Local\Temp\ipykernel_3100\3952101097.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['Estado', 'Capital']] = df.apply(lambda row: get_location_by_coordinates(row['Latitude'], row['Longitude']), axis=1)
C:\Users\surze\AppData\Local\Temp\ipykernel_3100\3952101097.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['Estado', 'Capital']] = df.apply(lambda row: get_location_by_coordinates(row['Latitude'], row['Longitude']), axis=1)


In [32]:
df.head()

,DatGeracaoConjuntoDados,NomAgente,IdeEmpresaProponenteProjeto,DscCodProjeto,DscTituloProjeto,DscTipologia,VlrCustoTotal,VlrRcbGlobal,VlrEnergiaEconomizadaTotal,VlrRetiradaDemandaPontaTotal,...,DscJustificativa,DatInicioProjeto,DatConclusaoProjeto,DscMetodologiaMv,Score,Address,Latitude,Longitude,Estado,Capital
1219,2024-04-01,ESPÍRITO SANTO DISTRIBUIÇÃO DE ENERGIA S/A.,693,PE-00380-0009/2006,Ind - CVRD,Industrial,797.58,NaN,2370.00,356.00,...,Diagnóstico Energético.,2005-01-01,2006-12-31,NaN,10.000000,EDP - Espírito Santo Distribuição de Energia S...,-18.094662,-40.517889,Espírito Santo,
2764,2024-04-01,COMPANHIA PAULISTA DE FORÇA E LUZ,1559,PE-00063-0058/2005,Educacional,Educacional,111250.00,NaN,126000.00,3738.00,...,NaN,2004-01-01,2005-12-31,NaN,4.045374,"Companhia Paulista de Força e Luz, Avenida Set...",-20.535621,-49.324210,São Paulo,
959,2024-04-01,ELEKTRO REDES S.A.,540,PE-00385-0010/2000,Eficientização Energética,Industrial,397253.78,0.36,397253.78,230.00,...,Não apresentou RF. Não são calculados o RIRP e...,1999-01-01,2000-12-31,NaN,3.607413,"Tecnologia Avanzada en Redes. S.A, Carretera a...",14.544281,-90.442728,Departamento de Guatemala,
1122,2024-04-01,ELEKTRO REDES S.A.,635,PE-00385-0017/2001,Eficientização Energética,Industrial,378229.26,0.36,378229.26,87.49,...,Não apresentou RF. A RCB uiliza 12%.,2000-01-01,2001-12-31,NaN,3.606494,"Tecnologia Avanzada en Redes. S.A, Carretera a...",14.544281,-90.442728,Departamento de Guatemala,
1051,2024-04-01,ELEKTRO REDES S.A.,596,PE-00385-0014/2006,Eficientização Energética,Industrial,577224.39,0.36,577224.39,128.61,...,Projeto a ser compensado no ciclo 2000/2001,2005-01-01,2006-12-31,NaN,3.606471,"Tecnologia Avanzada en Redes. S.A, Carretera a...",14.544281,-90.442728,Departamento de Guatemala,


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52 entries, 1219 to 170
Data columns (total 21 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   DatGeracaoConjuntoDados       52 non-null     datetime64[ns]
 1   NomAgente                     52 non-null     object        
 2   IdeEmpresaProponenteProjeto   52 non-null     int64         
 3   DscCodProjeto                 52 non-null     object        
 4   DscTituloProjeto              52 non-null     object        
 5   DscTipologia                  52 non-null     object        
 6   VlrCustoTotal                 52 non-null     float64       
 7   VlrRcbGlobal                  29 non-null     float64       
 8   VlrEnergiaEconomizadaTotal    52 non-null     float64       
 9   VlrRetiradaDemandaPontaTotal  52 non-null     float64       
 10  DscObjetivo                   4 non-null      object        
 11  DscJustificativa              28 no

In [34]:
df_unidos = pd.concat([df, df_null_address], ignore_index=True)

In [35]:
df_unidos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 21 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   DatGeracaoConjuntoDados       192 non-null    datetime64[ns]
 1   NomAgente                     192 non-null    object        
 2   IdeEmpresaProponenteProjeto   192 non-null    int64         
 3   DscCodProjeto                 192 non-null    object        
 4   DscTituloProjeto              192 non-null    object        
 5   DscTipologia                  192 non-null    object        
 6   VlrCustoTotal                 192 non-null    float64       
 7   VlrRcbGlobal                  113 non-null    float64       
 8   VlrEnergiaEconomizadaTotal    192 non-null    float64       
 9   VlrRetiradaDemandaPontaTotal  192 non-null    float64       
 10  DscObjetivo                   28 non-null     object        
 11  DscJustificativa              10

In [38]:
df_unidos['Score'].describe()

count    72.000000
mean      1.485795
std       1.323832
min       1.000000
25%       1.021201
50%       1.053785
75%       1.183299
max      10.000000
Name: Score, dtype: float64

In [36]:
df_unidos = df_unidos.dropna(subset=['Estado'])

In [37]:
df_unidos.to_csv(path+'df_unidos.csv')